# HW7

In [0]:
#!pip install tensorflow==1.4.0

     |████████████████████████████████| 41.2MB 87kB/s 
     |████████████████████████████████| 1.7MB 46.4MB/s 
     |████████████████████████████████| 890kB 37.5MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=3dec5b304818af4457e886306a9e662f437bf3f8fbf2f90403dfd79372aeb682
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
ERROR: stable-baselines 2.2.1 has requirement tensorflow>=1.5.0, but you'll have tensorflow 1.4.0 which is incompatible.
ERROR: magenta 0.3.19 has requirement tensorflow>=1.12.0, but you'll have tensorflow 1.4.0 which is incompatible.
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.0
    Uninstalling bleach-3.1.0:
      Successfully uninstalled bleach-3.1.0
  Found existing installation: tensorflow 1.15.0
    Uninst

In [0]:
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:472: FutureWarning: Passing (type, 1) or 

### Import data

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)  # y labels are oh-encoded

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
n_train = mnist.train.num_examples  # 55,000
n_validation = mnist.validation.num_examples  # 5000
n_test = mnist.test.num_examples  # 10,000

### Defining the Neural Network Architecture

In [0]:
n_input = 784  # input layer (28x28 pixels)
n_hidden1 = 512  # 1st hidden layer
n_hidden2 = 256  # 2nd hidden layer
n_hidden3 = 128  # 3rd hidden layer
n_output = 10  # output layer (0-9 digits)

In [0]:
learning_rate = 1e-4
n_iterations = 1000
batch_size = 128
dropout = 0.5

### Building the TensorFlow Graph

In [0]:
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])
keep_prob = tf.placeholder(tf.float32)

In [0]:
weights = {
    'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1)),
}

In [0]:
biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

In [0]:
layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
layer_drop = tf.nn.dropout(layer_3, keep_prob)
output_layer = tf.matmul(layer_drop, weights['out']) + biases['out']

In [0]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        labels=Y, logits=output_layer
        ))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

## Training and Testing

In [0]:
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [0]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [0]:
# train on mini batches
for i in range(n_iterations):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    sess.run(train_step, feed_dict={
        X: batch_x, Y: batch_y, keep_prob: dropout
        })

    # print loss and accuracy (per minibatch)
    if i % 100 == 0:
        minibatch_loss, minibatch_accuracy = sess.run(
            [cross_entropy, accuracy],
            feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0}
            )
        print(
            "Iteration",
            str(i),
            "\t| Loss =",
            str(minibatch_loss),
            "\t| Accuracy =",
            str(minibatch_accuracy)
            )

Iteration 0 	| Loss = 3.539604 	| Accuracy = 0.1328125
Iteration 100 	| Loss = 0.5423341 	| Accuracy = 0.8359375
Iteration 200 	| Loss = 0.41197228 	| Accuracy = 0.859375
Iteration 300 	| Loss = 0.39675936 	| Accuracy = 0.90625
Iteration 400 	| Loss = 0.43072298 	| Accuracy = 0.8671875
Iteration 500 	| Loss = 0.34667706 	| Accuracy = 0.8828125
Iteration 600 	| Loss = 0.39104363 	| Accuracy = 0.875
Iteration 700 	| Loss = 0.38350314 	| Accuracy = 0.90625
Iteration 800 	| Loss = 0.34282255 	| Accuracy = 0.890625
Iteration 900 	| Loss = 0.33652255 	| Accuracy = 0.90625


### 2. Test your accuracy using 3300 data points according to group number.

In [0]:
# We are group 7 using the first 3300 data points in test dataset
X_test = mnist.test.images[0:3300]
y_test = mnist.test.labels[0:3300]

In [0]:
test_accuracy = sess.run(accuracy, feed_dict={X: X_test, Y: y_test, keep_prob: 1.0})
print("\nAccuracy on test set:", test_accuracy)


Accuracy on test set: 0.89


### 3. Re-do the entire exercise but without using the dropouts from the final layer. Calculate accuracy of your new model and compare against your previously achieved accuracy. 

In [0]:
layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
output_layer = tf.matmul(layer_3, weights['out']) + biases['out']

In [0]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        labels=Y, logits=output_layer
        ))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [0]:
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [0]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [0]:
# Re-train the model without using dropout 
# train on mini batches
for i in range(n_iterations):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    sess.run(train_step, feed_dict={
        X: batch_x, Y: batch_y
        })

    # print loss and accuracy (per minibatch)
    if i % 100 == 0:
        minibatch_loss, minibatch_accuracy = sess.run(
            [cross_entropy, accuracy],
            feed_dict={X: batch_x, Y: batch_y}
            )
        print(
            "Iteration",
            str(i),
            "\t| Loss =",
            str(minibatch_loss),
            "\t| Accuracy =",
            str(minibatch_accuracy)
            )

Iteration 0 	| Loss = 3.805387 	| Accuracy = 0.078125
Iteration 100 	| Loss = 0.5196793 	| Accuracy = 0.875
Iteration 200 	| Loss = 0.52463174 	| Accuracy = 0.859375
Iteration 300 	| Loss = 0.32973155 	| Accuracy = 0.9140625
Iteration 400 	| Loss = 0.26264486 	| Accuracy = 0.9140625
Iteration 500 	| Loss = 0.2869534 	| Accuracy = 0.921875
Iteration 600 	| Loss = 0.3160813 	| Accuracy = 0.921875
Iteration 700 	| Loss = 0.43384707 	| Accuracy = 0.8984375
Iteration 800 	| Loss = 0.3858138 	| Accuracy = 0.8984375
Iteration 900 	| Loss = 0.34583494 	| Accuracy = 0.90625


In [0]:
test_accuracy_2 = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
print("\nAccuracy on test set:", test_accuracy_2)


Accuracy on test set: 0.89454544


## Test on a single image

In [0]:
import numpy as np
from PIL import Image
from google.colab import files

In [0]:
uploaded = files.upload()

Saving test_img.png to test_img (1).png


In [0]:
img = np.invert(Image.open("test_img.png").convert('L')).ravel()

In [0]:
prediction = sess.run(tf.argmax(output_layer, 1), feed_dict={X: [img]})
print ("Prediction for test image:", np.squeeze(prediction))

Prediction for test image: 2


### 5. Use a multinomial logistic regression model to achieve the same handwriting identification task and compare the out-of-sample accuracy against the results of the neural network.

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [0]:
X_train = mnist.train.images
y_train = mnist.train.labels

In [0]:
X_train.shape

(55000, 784)

In [0]:
y_train.shape

(55000, 10)

In [0]:
# Convert y to a list with 10 levels
y_train_convert = []
for i in range(y_train.shape[0]):
  for j in range(y_train.shape[1]):
    if y_train[i][j] == 1.0:
      y_train_convert.append(j)

In [0]:
len(y_train_convert)

55000

In [0]:
# Fit multinomial logistic regression on training data (without penalty)
lr = LogisticRegression(penalty = 'none', random_state = 20, multi_class = 'multinomial').fit(X_train, y_train_convert)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
y_pred = lr.predict(X_test)

In [0]:
y_test_convert = []
for i in range(y_test.shape[0]):
  for j in range(y_test.shape[1]):
    if y_test[i][j] == 1.0:
      y_test_convert.append(j)

In [0]:
accuracy_score(y_test_convert, y_pred)

0.9063636363636364

In [0]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_test_convert, y_pred)

array([[[2982,   20],
        [   4,  294]],

       [[2904,   20],
        [   7,  369]],

       [[2924,   28],
        [  36,  312]],

       [[2908,   45],
        [  38,  309]],

       [[2925,   24],
        [  30,  321]],

       [[2958,   38],
        [  50,  254]],

       [[2972,   23],
        [  26,  279]],

       [[2928,   31],
        [  35,  306]],

       [[2950,   35],
        [  50,  265]],

       [[2940,   45],
        [  33,  282]]])

In [0]:
# Fit lasso multinomial logistic regression on training data
lr_lasso = LogisticRegression(penalty = 'l2', random_state = 20, multi_class = 'multinomial').fit(X_train, y_train_convert)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
y_pred_lasso = lr_lasso.predict(X_test)

In [0]:
accuracy_score(y_test_convert, y_pred_lasso)

0.9066666666666666

In [0]:
multilabel_confusion_matrix(y_test_convert, y_pred_lasso)

array([[[2980,   22],
        [   4,  294]],

       [[2904,   20],
        [   7,  369]],

       [[2925,   27],
        [  35,  313]],

       [[2910,   43],
        [  38,  309]],

       [[2926,   23],
        [  28,  323]],

       [[2960,   36],
        [  50,  254]],

       [[2972,   23],
        [  25,  280]],

       [[2929,   30],
        [  36,  305]],

       [[2949,   36],
        [  51,  264]],

       [[2937,   48],
        [  34,  281]]])